# LifetimeFit

### Author: Till Leissner (till@mci.sdu.dk)


This script is designed to fit TCSPC data.

In [ ]:
# Import modules
import numpy as np
import pandas as pd
import importlib
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import plotly.express as px
import plotly.tools as tls
from pandas.plotting import table 
import matplotlib.pyplot as plt
from lmfit import Parameters, Parameter

import _ltfit_ as lt

# Set general options for plots and tables
pd.set_option('max_colwidth',50)
pd.options.display.float_format = '{:.3f}'.format


## Load your data

You need to define the path to the folder that contains your data. 
Currently only "dac" files from Hamamatsu HPD-TA are accepted.

In [ ]:
#%%capture
### Import data using dac files
datapath = './data' #
datafilter = '16*.dac' # You can filter your data files
irffile = "irf/1ns.dac" # File containing irf data

In [ ]:
# Importing data and return a dict with TCSPC trace, decay curve (integrated), spectrum (integrated)
data = lt._load_datasets_(datapath, datafilter)
data = lt._init_datasets_(data)

In [ ]:
# Plot the data
fig = lt._plot_datasets_(data)


In [ ]:
# Select your data range for fitting
# When you want to use idividual start/stop limit for each dataset, you can also define a arrays.

importlib.reload(_ltfit_)
#Normally the default values work pretty well
options = dict()
options['tstart_offset'] = 10            # Skip this number of data points right of PL peak (to exclude scattering etc.)
options['threshold']     = 10            # Datapoints below this number of count will be skipped in the fit

#Please adjust these values
options['lstart']        = [730]        # Lower wavelength limit (in nm), set [0] for full range, [780,890,...] for individual limits for each sample
options['lstop']         = [770]        # Upper wavelength limit (in nm), set [0] for full range, [780,890,...] for individual limits for each sample
options['tstart']        = [0]          # Start time for fits (in ns), set [0] for automatic detection, [1,2,...] for individual limits for each sample
options['tstop']         = [180]          # Upper time limit (in ns), set [0] for full range, [5,7,...] for individual limits for each sample
options['exponents']     = 2            # Number of exponents used to fit datasets
options['convolution']   = False         # Do deconvolution with IRF: True/False

### Check and update options - do not change
data = lt._check_options_(data, options) # ensures correct format of input
data = lt._init_fitdata_(data)
data = lt._init_parameters_(data)
if options['convolution']:
    data=lt._fit_irf_(data,irffile)
    

In [ ]:
### You can overwrite parameters for each sample ('key') if needed here:
## Examples:

for key in data:
#    params = Parameters()   
    #           (Name,  Value,  Vary,   Min,  Max,  Expr)
#    params.add_many(
#                ('a1',   4000,  True,  1e2, 1e5,  None), 
#                ('tau1',    0.03,  True,  0.01, 0.1,  None),
#                ('a2',   800,  True,  1e2, 1e5,  None), 
#                ('tau2',    0.166,  True,  0.1, 1,  None),
#                ('x0',    0.1,  True,  -0.15, 0.15,  None), # will be fixed later to maximum position + toffset
#                ('y0',   4,  True,   0  , 50,  None), # will be fixed later to y_offset of tail
#                ('tau', 0, False, 0, 1000, 'a1/(a1+a2)*tau1+a2/(a1+a2)*tau2')
#                )
#    data[key]['init_params'] = params

    data[key]['init_params']['x0'] = Parameter(name='x0', value=0, vary=False, min=-100, max=100)
    #data[key]['init_params']['y0'] = Parameter(name='y0', vary=True, value=0,min=1000, max=5000)
    #data[key]['init_params']['tau1'] = Parameter(name='tau1', value=1, max=40)
    #data[key]['init_params']['tau2'] = Parameter(name='tau2', value=20, max=100)
    #data[key]['init_params']['a2'] = Parameter(name='a2', value=1000, max=10000)
    #data[key]['init_params']['a3'] = Parameter(name='a3', value=1000, max=10000)

## Perform fit

In [ ]:
importlib.reload(_ltfit_)
data = lt._do_fitting_(data, show=False)

In [ ]:
# Visualize fits and data
importlib.reload(_ltfit_)
fig = lt._show_fitresults_(data)
fig.update_yaxes(range=[0, 6], row=1, col=1)
#fig.update_yaxes(type="linear", row=1,col=1)
fig.update_layout(title=('Wavelength:' +str(options['lstart'][0])+' - '+str(options['lstop'][0])+' nm'))
fig

In [ ]:
dfexp = lt._to_df_(data)
dfexp

In [ ]:
import os
import datetime
path_out = os.path.join(datapath)
prefix=('Wavelength_' +str(options['lstart'][0])+'-'+str(options['lstop'][0])+'nm')
fig.write_html(os.path.join(path_out,prefix+'_TRPL_results_n='+str(options['exponents'])+'_conv='+str(options['convolution'])+'_figure.html'))
dfexp.to_html(os.path.join(path_out,prefix+'_TRPL_results_n='+str(options['exponents'])+'_conv='+str(options['convolution'])+'.html'))
#!jupyter nbconvert --to html_embed *.ipynb 

In [ ]:
import importlib
fig2 = lt._plot_spectra_(data)


In [ ]:
fig2 = lt._plot_decaycurves_(data)

In [ ]:
#Export data and fit results as txt files (creates a file for each dataset)
for key in data:
    df = pd.DataFrame({'data': data[key]['fitdata'], 'fit':data[key]['fitresult'].best_fit})
    df.to_csv(os.path.join(path_out,prefix+key+'.fit'),index=True, header=True, decimal=',', sep=' ', float_format='%.3f')